In [60]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

In [61]:
# Import the OHLCV dataset into a Pandas Dataframe
NFL_df = pd.read_csv(
    Path("./Data-Table 1.csv"))

# Review the DataFrame
NFL_df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,NaN,Y,Y,1.83,1.74,...,51.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,NaN,Y,NaN,1.87,1.77,...,48.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,NaN,Y,NaN,1.77,1.65,...,45.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,NaN,Y,NaN,1.50,1.48,...,46.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,NaN,Y,NaN,1.52,1.38,...,48.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN


In [62]:
home_team_row = NFL_df["Home Team"]
away_team_row = NFL_df["Away Team"]

In [63]:
def game_winner(Home_Team, Away_Team):
    if Home_Score > Away_Score:
        return "Home Team"
    elif Home_Score < Away_Score:
        return "Away Score"
    else:
        return 'Tie'

In [64]:
winner = []
for i, row in NFL_df.iterrows():
    if row['Home Score'] > row['Away Score']:
        winner.append(row['Home Team'])
    elif row['Away Score'] > row['Home Score']:
        winner.append(row['Away Team'])
    else:
        winner.append('Tie')

In [65]:
NFL_df['winner'] = winner

In [66]:
NFL_df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,...,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes,winner
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,NaN,Y,Y,1.83,1.74,...,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN,Kansas City Chiefs
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,NaN,Y,NaN,1.87,1.77,...,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN,Kansas City Chiefs
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,NaN,Y,NaN,1.77,1.65,...,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN,Philadelphia Eagles
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,NaN,Y,NaN,1.50,1.48,...,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN,San Francisco 49ers
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,NaN,Y,NaN,1.52,1.38,...,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,NaN,Cincinnati Bengals


In [67]:
#Drop Unused columns
NFL_odds_df = NFL_df.drop(columns = ["Overtime?", "Playoff Game?", "Neutral Venue?", "Notes", "Total Score Open", "Total Score Min", "Total Score Close",
               "Total Score Over Open","Total Score Over Min", "Total Score Over Max", "Total Score Over Close", "Total Score Under Open", 
               "Total Score Under Min" , "Total Score Under Max", "Total Score Under Close", "Total Score Max", "Date", "winner"])

In [68]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,Philadelphia Eagles,Kansas City Chiefs,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,Kansas City Chiefs,Cincinnati Bengals,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,Philadelphia Eagles,San Francisco 49ers,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,San Francisco 49ers,Dallas Cowboys,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,Buffalo Bills,Cincinnati Bengals,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [69]:
NFL_odds_df.dtypes

Home Team                object
Away Team                object
Home Score                int64
Away Score                int64
Home Odds Open          float64
Home Odds Min           float64
Home Odds Max           float64
Home Odds Close         float64
Away Odds Open          float64
Away Odds Min           float64
Away Odds Max           float64
Away Odds Close         float64
Home Line Open          float64
Home Line Min           float64
Home Line Max           float64
Home Line Close         float64
Away Line Open          float64
Away Line Min           float64
Away Line Max           float64
Away Line Close         float64
Home Line Odds Open     float64
Home Line Odds Min      float64
Home Line Odds Max      float64
Home Line Odds Close    float64
Away Line Odds Open     float64
Away Line Odds Min      float64
Away Line Odds Max      float64
Away Line Odds Close    float64
dtype: object

In [70]:
X_scaler = StandardScaler()

In [71]:
encoder = OneHotEncoder(sparse=False)

In [72]:
categorical_variables = list(NFL_odds_df.dtypes[NFL_odds_df.dtypes == "object"].index)

In [73]:
encoded_data = encoder.fit_transform(NFL_odds_df[categorical_variables])

In [74]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns = encoder.get_feature_names(categorical_variables)
)

/Users/tylergoering/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [75]:
encoded_df = pd.concat(
    [
        NFL_odds_df,
        encoded_df
    ],
    axis=1
)

In [76]:
encoded_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Team_Pittsburgh Steelers,Away Team_San Diego Chargers,Away Team_San Francisco 49ers,Away Team_Seattle Seahawks,Away Team_St. Louis Rams,Away Team_Tampa Bay Buccaneers,Away Team_Tennessee Titans,Away Team_Washington Commanders,Away Team_Washington Football Team,Away Team_Washington Redskins
0,Philadelphia Eagles,Kansas City Chiefs,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Kansas City Chiefs,Cincinnati Bengals,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Philadelphia Eagles,San Francisco 49ers,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,San Francisco 49ers,Dallas Cowboys,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Buffalo Bills,Cincinnati Bengals,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
NFL_odds_df['Home Team'] = NFL_odds_df['Home Team'].fillna(int(0))

In [78]:
NFL_odds_df['Away Team'] = NFL_odds_df['Away Team'].fillna(int(1))

In [79]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,Philadelphia Eagles,Kansas City Chiefs,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,Kansas City Chiefs,Cincinnati Bengals,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,Philadelphia Eagles,San Francisco 49ers,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,San Francisco 49ers,Dallas Cowboys,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,Buffalo Bills,Cincinnati Bengals,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91


In [80]:
binary_winner = []
for i, row in NFL_odds_df.iterrows():
    if row['Home Score'] > row['Away Score']:
        winner.append(row['Home Team'])
    elif row['Away Score'] > row['Home Score']:
        winner.append(row['Away Team'])
    else:
        winner.append('Tie')

In [81]:
NFL_odds_df.head()

,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,...,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close
0,Philadelphia Eagles,Kansas City Chiefs,35,38,1.83,1.74,1.83,1.83,2.00,2.00,...,2.5,1.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
1,Kansas City Chiefs,Cincinnati Bengals,23,20,1.87,1.77,2.20,1.77,1.95,1.71,...,2.0,2.0,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
2,Philadelphia Eagles,San Francisco 49ers,31,7,1.77,1.65,1.77,1.65,2.10,2.10,...,3.0,3.0,1.87,2.00,1.87,2.00,1.95,1.95,1.83,1.83
3,San Francisco 49ers,Dallas Cowboys,19,12,1.50,1.48,1.54,1.50,2.70,2.65,...,4.0,3.5,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91
4,Buffalo Bills,Cincinnati Bengals,10,27,1.52,1.38,1.52,1.38,2.67,2.67,...,6.0,6.0,1.95,1.91,1.95,1.91,1.87,1.87,1.91,1.91
